Setup the environment here

In [3]:
import pandas as pd

Filter the test file for the negative reviews in the local files area

In [6]:
df = pd.read_csv("amazon_review_polarity_csv/test.csv", names = ["polarity", "title", "review"])
df = df[df.polarity == 1]

Load the output into S3 in prep for comprehend processing

In [9]:
test = df.drop(columns = ["title", "polarity"])
test.to_csv("test_file_10k_sample_only_content.csv", encoding="utf-8")
test = test.sample(10000)

Used the following to upload via terminal:
aws s3 cp test_file_10k_sample_only_content.csv s3://nlp-jsears-output/test_file_10k_sample_only_content.csv

Process using comprehend

In [11]:
import json
import csv
from bson import json_util
import boto3

comprehend = boto3.client('comprehend')
input_s3_url = "s3://nlp-jsears-output/test_file_10k_sample_only_content.csv"
input_doc_format = "ONE_DOC_PER_LINE"
#output_s3_url = "s3://nlp-kc-learning/js-output/"
output_s3_url = "s3://nlp-jsears-output/output"
#data_access_role_arn = "arn:aws:iam::023375022819:role/service-role/AmazonSageMaker-ExecutionRole-20181029T121824"
data_access_role_arn = "arn:aws:iam::023375022819:role/service-role/AmazonComprehendServiceRole-mark"
number_of_topics = 10

input_data_config = {"S3Uri": input_s3_url, "InputFormat": input_doc_format}
output_data_config = {"S3Uri": output_s3_url}

start_topics_detection_job_result = \
    comprehend.start_topics_detection_job(NumberOfTopics=number_of_topics,
                                          JobName="jsears_test_topics_1",
                                          InputDataConfig=input_data_config,
                                          OutputDataConfig=output_data_config,
                                          DataAccessRoleArn=data_access_role_arn)

print('start_topics_detection_job_result: ' + json.dumps(start_topics_detection_job_result))
 
job_id = start_topics_detection_job_result["JobId"]
 
print('job_id: ' + job_id)
 
describe_topics_detection_job_result = comprehend.describe_topics_detection_job(JobId=job_id)
 
print('describe_topics_detection_job_result: ' + json.dumps(describe_topics_detection_job_result, default=json_util.default))

list_topics_detection_jobs_result = comprehend.list_topics_detection_jobs()
 
print('list_topics_detection_jobs_result: ' + json.dumps(list_topics_detection_jobs_result, default=json_util.default))

start_topics_detection_job_result: {"JobId": "f2a13706eac6c79237e4f3c52f4cda95", "JobStatus": "SUBMITTED", "ResponseMetadata": {"RequestId": "cc8b265e-6770-11e9-a3a7-0fc2d380022a", "HTTPStatusCode": 200, "HTTPHeaders": {"x-amzn-requestid": "cc8b265e-6770-11e9-a3a7-0fc2d380022a", "content-type": "application/x-amz-json-1.1", "content-length": "68", "date": "Thu, 25 Apr 2019 15:42:57 GMT"}, "RetryAttempts": 0}}
job_id: f2a13706eac6c79237e4f3c52f4cda95
describe_topics_detection_job_result: {"TopicsDetectionJobProperties": {"JobId": "f2a13706eac6c79237e4f3c52f4cda95", "JobName": "jsears_test_topics_1", "JobStatus": "SUBMITTED", "SubmitTime": {"$date": 1556206977766}, "InputDataConfig": {"S3Uri": "s3://nlp-jsears-output/test_file_10k_sample_only_content.csv", "InputFormat": "ONE_DOC_PER_LINE"}, "OutputDataConfig": {"S3Uri": "s3://nlp-jsears-output/output/023375022819-TOPICS-f2a13706eac6c79237e4f3c52f4cda95/output/output.tar.gz"}, "NumberOfTopics": 10, "DataAccessRoleArn": "arn:aws:iam::0233